In [113]:
import pandas as pd
import os

In [114]:
file_path="./"

In [115]:
emailDF=pd.read_excel(os.path.join(file_path,"chinesespam.xlsx"))

In [116]:
emailDF.head(3)

,type,text
0,ham,1506讲的是孔子后人的故事。一个老领导回到家乡，跟儿子感情不和，跟贪财的孙子孔为本和睦。老...
1,ham,那他为什么不愿意起诉，既然这样了！起诉后也有充分的理由！MM莫不是还生活在电影中，个人认为这...
2,ham,我觉得，负债不要紧，最重要的是能负得起这个责任来，\n欠了那么多钱，至少对当初拿出爱心来的网...


In [117]:
# 查看垃圾邮件的数量
emailDF[emailDF['type'] == "spam"].shape[0]

50

In [118]:
# 查看正常邮件的数量
emailDF[emailDF['type'] == "ham"].shape[0]

100

In [119]:
import codecs
# 加载stopwords
stopwords = codecs.open(os.path.join(file_path, 'stopwords.txt'), 'r', 'UTF8').read().split('\n')

In [144]:
# 分词
import jieba
for i in range(emailDF.shape[0]):
    text=emailDF.iloc[i]["text"]
    words=[]
    seg_list=jieba.cut(text)
    for seg in seg_list:
        #过滤掉空字符串和stopwords,也可以在CountVectorizer引入过滤stopwords参数
        if (seg.isalpha()):
            words.append(seg)
        #拼接分词后的结果并写回原数据集
        emailDF.iloc[i]["text"]=" ".join(words)

In [145]:
emailDF.head(3)

,type,text
0,ham,讲 孔子 后人 故事 领导 回到 家乡 儿子 感情 贪财 孙子 孔为 和睦 领导 弟弟 魏宗...
1,ham,起诉 起诉 理由 MM 莫不是 生活 电影 中 结婚 感情 感情 何来 感情 传统 家庭 责...
2,ham,负债 不要紧 负得起 责任 欠 多钱 至少 当初 拿出 爱心 网友 交待 心 实在 能力 一...


In [146]:
# 将词语转为向量的形式 (onehot 词袋模型)
from sklearn.feature_extraction.text import CountVectorizer
def words2Matrix(text):
    # 不做二值化 
    #ngram_range=(1,1) 表示 unigram, ngram_range=(2,2) 表示 bigram, ngram_range=(3,3) 表示 thirgram
    #stop_words 过滤停用词
    cv = CountVectorizer(binary=False,ngram_range=(1,1),stop_words=stopwords)
    vector=cv.fit_transform(text)
    result = pd.DataFrame(vector.toarray())
    # 获取列名
    keys = []
    values = []
    for key,value in cv.vocabulary_.items():
        keys.append(key)
        values.append(value)
    df = pd.DataFrame(data = {"key" : keys, "value" : values})
    colnames = df.sort_values("value")["key"].values
    result.columns = colnames
    return result

In [147]:
wordsMatrix=words2Matrix(emailDF["text"])

/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'aren', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'lex', 'll', 'mon', 'null', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn', '１２', 'ｌｉ', 'ｚｘｆｉｔｌ'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [148]:
wordsMatrix.shape

(150, 5891)

In [149]:
# 去除稀疏数据的列
cnt=0
for col_name in wordsMatrix.columns:
    # 如果缺失率超过百分之十
    if (wordsMatrix[col_name].shape[0] - wordsMatrix[col_name][wordsMatrix[col_name]==0].count())<=15:
        # 删除列
        cnt=cnt+1
        wordsMatrix=wordsMatrix.drop(col_name,axis=1)
print(cnt)

5844


In [150]:
wordsMatrix.shape

(150, 47)

In [151]:
wordsMatrix.head(3)

,gg,mm,www,一年,一点,上网,不到,不好,事情,代开,...,电脑,电话,经理,联系人,负责人,财务,运输,进一步,进项,郑重
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [164]:
# 切分训练集和测试集
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(wordsMatrix,emailDF["type"],test_size=0.2)
from sklearn import naive_bayes as nb
model=nb.BernoulliNB(alpha=1,binarize=True)
model.fit(X_train,y_train)
print("伯努利朴素贝叶斯得分:%s"%model.score(X_test,y_test))
model.score(X_test,y_test)

mnbMolde=nb.MultinomialNB(alpha=1)
mnbMolde.fit(X_train,y_train)
mnbMolde.score(X_test,y_test)
print("多项式朴素贝叶斯得分:%s"%mnbMolde.score(X_test,y_test))

gnbModel=nb.GaussianNB()
gnbModel.fit(X_train,y_train)
gnbModel.score(X_test,y_test)
print("高斯朴素贝叶斯得分:%s"%gnbModel.score(X_test,y_test))


伯努利朴素贝叶斯得分:0.9
多项式朴素贝叶斯得分:0.9
高斯朴素贝叶斯得分:0.9


In [155]:
model.predict(X_test)

array(['spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam',
       'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham',
       'spam', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'spam',
       'ham', 'ham', 'ham'], dtype='<U4')

In [141]:
y_test

57      ham
8       ham
14      ham
52      ham
149    spam
51      ham
124    spam
130    spam
0       ham
72      ham
67      ham
74      ham
90      ham
97      ham
22      ham
61      ham
118    spam
34      ham
86      ham
58      ham
88      ham
39      ham
6       ham
89      ham
142    spam
126    spam
21      ham
33      ham
148    spam
92      ham
Name: type, dtype: object

In [142]:
help(CountVectorizer)

Help on class CountVectorizer in module sklearn.feature_extraction.text:

class CountVectorizer(sklearn.base.BaseEstimator, VectorizerMixin)
 |  CountVectorizer(input='content', encoding='utf-8', decode_error='strict', strip_accents=None, lowercase=True, preprocessor=None, tokenizer=None, stop_words=None, token_pattern='(?u)\\b\\w\\w+\\b', ngram_range=(1, 1), analyzer='word', max_df=1.0, min_df=1, max_features=None, vocabulary=None, binary=False, dtype=<class 'numpy.int64'>)
 |  
 |  Convert a collection of text documents to a matrix of token counts
 |  
 |  This implementation produces a sparse representation of the counts using
 |  scipy.sparse.csr_matrix.
 |  
 |  If you do not provide an a-priori dictionary and you do not use an analyzer
 |  that does some kind of feature selection then the number of features will
 |  be equal to the vocabulary size found by analyzing the data.
 |  
 |  Read more in the :ref:`User Guide <text_feature_extraction>`.
 |  
 |  Parameters
 |  ----------